In [1]:
import sqlite3
from sqlalchemy import create_engine, text

# Establishing a connection to SQLite database
conn = sqlite3.connect('example.db')
c = conn.cursor()

# Using SQLAlchemy for more advanced operations
engine = create_engine('sqlite:///example.db')

# Function to execute SQL queries
def execute_query(query):
    with engine.connect() as connection:
        result = connection.execute(text(query))
        return result

# Function to print the result of a query
def print_query_result(query):
    result = execute_query(query)
    for row in result:
        print(row)

# Transaction Control Language (TCL) commands
def tcl_commands():
    try:
        # Start transaction
        c.execute("BEGIN TRANSACTION;")
        # Example operations
        c.execute("INSERT INTO students (name, age) VALUES ('John Doe', 25);")
        c.execute("INSERT INTO students (name, age) VALUES ('Jane Doe', 22);")
        c.execute("INSERT INTO students (name, age) VALUES ('Alice', 30);")
        # Commit transaction
        conn.commit()
        print("Transaction committed successfully.")
    except Exception as e:
        # Rollback in case of error
        conn.rollback()
        print(f"Transaction failed: {e}")

# Creating a Trigger
def create_trigger():
    trigger_query = '''
    CREATE TRIGGER IF NOT EXISTS before_insert_student
    BEFORE INSERT ON students
    FOR EACH ROW
    BEGIN
        SELECT CASE
            WHEN NEW.age < 0 THEN
                RAISE (ABORT, 'Age cannot be negative')
        END;
    END;
    '''
    execute_query(trigger_query)
    print("Trigger created successfully.")

# Creating a View
def create_view():
    view_query = '''
    CREATE VIEW IF NOT EXISTS student_ages AS
    SELECT name, age
    FROM students;
    '''
    execute_query(view_query)
    print("View created successfully.")

# Adding more complex operations
def complex_operations():
    # Update operation
    c.execute("UPDATE students SET age = age + 1 WHERE name = 'John Doe';")
    conn.commit()
    
    # Delete operation
    c.execute("DELETE FROM students WHERE name = 'Alice';")
    conn.commit()
    
    print("Complex operations executed successfully.")

# Dropping a View
def drop_view():
    drop_view_query = '''
    DROP VIEW IF EXISTS student_ages;
    '''
    execute_query(drop_view_query)
    print("View dropped successfully.")

# Dropping a Trigger
def drop_trigger():
    drop_trigger_query = '''
    DROP TRIGGER IF EXISTS before_insert_student;
    '''
    execute_query(drop_trigger_query)
    print("Trigger dropped successfully.")

# Additional function to insert data
def insert_data(name, age):
    try:
        c.execute("INSERT INTO students (name, age) VALUES (?, ?);", (name, age))
        conn.commit()
        print(f"Inserted {name}, age {age}.")
    except Exception as e:
        conn.rollback()
        print(f"Failed to insert data: {e}")

# Additional function to display all students
def display_all_students():
    print("All students:")
    print_query_result("SELECT * FROM students;")

# Main execution
def main():
    # Create table for demonstration
    c.execute('''
    CREATE TABLE IF NOT EXISTS students (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        age INTEGER NOT NULL
    );
    ''')

    # TCL Commands
    tcl_commands()

    # Create Trigger
    create_trigger()

    # Create View
    create_view()

    # Query and display data from the view
    print("Students data from view:")
    print_query_result("SELECT * FROM student_ages;")

    # Perform complex operations
    complex_operations()

    # Query and display updated data from the view
    print("Updated students data from view:")
    print_query_result("SELECT * FROM student_ages;")

    # Insert additional data
    insert_data('Bob', 27)
    insert_data('Charlie', -1)  # This should fail due to the trigger

    # Display all students
    display_all_students()

    # Drop the view
    drop_view()

    # Drop the trigger
    drop_trigger()

if __name__ == "__main__":
    main()


Transaction committed successfully.
Trigger created successfully.
View created successfully.
Students data from view:
('John Doe', 27)
('Jane Doe', 22)
('John Doe', 26)
('Jane Doe', 22)
('Bob', 27)
('John Doe', 25)
('Jane Doe', 22)
('Alice', 30)
Complex operations executed successfully.
Updated students data from view:
('John Doe', 28)
('Jane Doe', 22)
('John Doe', 27)
('Jane Doe', 22)
('Bob', 27)
('John Doe', 26)
('Jane Doe', 22)
Inserted Bob, age 27.
Failed to insert data: Age cannot be negative
All students:
(1, 'John Doe', 28)
(2, 'Jane Doe', 22)
(4, 'John Doe', 27)
(5, 'Jane Doe', 22)
(7, 'Bob', 27)
(8, 'John Doe', 26)
(9, 'Jane Doe', 22)
(11, 'Bob', 27)
View dropped successfully.
Trigger dropped successfully.
